In [1]:
import numpy as np
import pandas as pd
from pyDOE2 import *

# Recursos
- [Catapulta](https://sigmazone.com/catapult/)
- [Documentação pyDOE2](https://pythonhosted.org/pyDOE/)
- [Tutorial de DOE com catapulta do Sigmazone](https://sigmazone.com/quantum_xl_doe_tutorial/#Step_1:_Setting_up_the_Designed_Experiment)
- [Response Optimization with Design of Experiments and python](https://towardsdatascience.com/response-optimization-with-design-of-experiments-and-python-63f9afb3f26f)
- [Youtube: Design of Experiments (DOE) - Minitab Masters Module 5](https://www.youtube.com/watch?v=rXmrMu-czM4)
- [Curso de Design of Experiments da PennState](https://online.stat.psu.edu/stat503/lesson/1)

## Obtendo as amostras
Devido ao número de fatores e níveis, optamos por usar o Latin Hypercube Sampling (LHS) para obter amostras do nosso espaço. 

O Latin Hypercube Sampling (LHS) é uma abordagem que divide a faixa de cada fator de entrada em intervalos iguais e garante que cada intervalo seja representado exatamente por uma amostra. O LHS é frequentemente utilizado para explorar o espaço de design de forma mais eficiente do que a amostragem aleatória simples.

O código abaixo só precisou ser rodado uma vez para a obtenção do dataframe com as configurações do experimento.

In [2]:
# Define fatores e número de amostras
factor_names = ['release_angle', 'firing_angle', 'cup_elevation', 'bungee_position', 'pin_elevation']
num_factors = len(factor_names)
num_samples = 70

# Define valores máximos e mínimos para cada fator.
min_values = [90, 90, 200, 100, 100]
max_values = [185, 140, 300, 200, 200]

# # Cria as amostras
# samples = lhs(num_factors, samples=num_samples)

# # Coloca as amostras nos intervalos das configurações
# scaled_samples = min_values + samples * (np.array(max_values) - np.array(min_values))

# # Converte os valores para int
# scaled_samples = scaled_samples.astype(int)

# # Transforma as amostras em um dataframe
# df = pd.DataFrame(scaled_samples)
# df.columns = factor_names

In [3]:
# Descarta linha se release_angle < firing_angle
# for index, row in df.iterrows():
#     if (row['release_angle'] < row['firing_angle']):
#         df = df.drop(index)
# df = df.reset_index(drop=True)
# df["distance"] = ""
# df.to_csv("df_catapulta_empty.csv")

## Realizando o experimento.
Colocamos as configurações de cada linha do dataframe na [catapulta](https://sigmazone.com/catapult/) para preencher a coluna `distance` e salvá-lo como um novo dataframe.

A planilha compartilhada está disponível no [Google Sheets](https://docs.google.com/spreadsheets/d/1e-EHZcRrpu5jy7baX_Z1_XDp0cppIyoXXveRIzQKk2E/edit#gid=1081657349)

In [4]:
df = pd.read_csv("df_catapulta_complete.csv")
# df = df.dropna(ignore_index=True)
df = df.dropna(ignore_index=True).drop('Unnamed: 0', axis=1)
df['distance'] = df['distance'].astype(int)
df

,release_angle,firing_angle,cup_elevation,bungee_position,pin_elevation,distance
0,183,100,211,168,120,185
1,173,124,240,110,100,125
2,128,104,243,117,163,96
3,148,131,273,135,138,82
4,123,90,265,194,118,105
5,148,116,254,153,106,136
6,178,137,230,131,141,145
7,118,91,288,179,136,107
8,150,94,229,128,195,149
9,169,114,233,189,192,341


## Análise dos dados
Usamos a biblioteca `pyDOE2` para realizar a análise de variância, entre outras.

## Otimização
Técnica de Análise de Superfície de Resposta (RSM) para achar a melhor configuração da catapulta.

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize

In [14]:
# Variáveis independentes
x = df[['release_angle', 'firing_angle','cup_elevation', 'bungee_position', 'pin_elevation']]

# Variável de resposta
y = df['distance']

# Divide dataset em variáveis de treinamento e de teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=42)

Ao utilizar a técnica de análise de superfície de resposta (RSM), transformar as variáveis de teste em um polinômio permite capturar relações não lineares entre os fatores e a resposta. É uma forma de expandir o espaço de recursos e levar em consideração possíveis interações e efeitos não lineares entre as variáveis.

In [15]:
poly = PolynomialFeatures(degree=2)
x_train_poly = poly.fit_transform(x_train)
x_test_poly = poly.fit_transform(x_test)

In [16]:
# Treina o modelo
model = LinearRegression()
model.fit(x_train_poly, y_train)

# Predições
y_pred = model.predict(x_test_poly)

In [17]:
# Avalia o desempenho do modelo. Quanto menor o MSE, melhor.
mse = mean_squared_error(y_test, y_pred)
mse

507.0164988875906